# 마인크래프트 예측보고서

## 세션 생성 및 데이터 불러오기

In [1]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import col, when, sum as spark_sum
from pyspark.ml.regression import LinearRegression
from pyspark.ml.feature import VectorAssembler, StringIndexer, OneHotEncoder, StandardScaler
from pyspark.ml import Pipeline

# Spark 세션 생성
spark = SparkSession.builder.appName("MobDropValueRegression").getOrCreate()

# 데이터 불러오기
mobs = spark.read.csv('../learning_spark_data/minecraft/Mobs.csv', header=True, inferSchema=True)
food = spark.read.csv('../learning_spark_data/minecraft/Food.csv', header=True, inferSchema=True)
mob_food = spark.read.csv('../learning_spark_data/minecraft/MobFoodDrops.csv', header=True, inferSchema=True)

## 필요없는 데이터 컬럼 drop

In [2]:
mobs = mobs.drop("behaviorTypes", "spawnBehavior", "debutDate", "minecraftVersion", "reproductiveRequirement")
food = food.drop("debutDate", "minecraftVersion")

## NULL 처리 및 캐스팅

In [3]:
mobs = mobs.withColumn("healthPoints", col("healthPoints").cast("double"))
mobs = mobs.withColumn("maxDamage", when(col("maxDamage").isNull(), 0).otherwise(col("maxDamage")).cast("double"))
mobs = mobs.withColumn("difficultyScore", col("healthPoints") + col("maxDamage") * 2)

food = food.withColumn("hunger", col("hunger").cast("double"))  # 드롭 가치 = hunger

- healthPoints: 체력을 숫자(double)로 바꿉니다.
- maxDamage: 공격력이 비어 있을 경우 0으로 채우고 숫자로 변환합니다.
- difficultyScore: 난이도를 계산하는 사용자 정의 지표입니다.
- hunger: 음식의 포만감 점수를 숫자로 변환합니다.

---

-  왜 difficultyScore = healthPoints + maxDamage * 2 인가요?
    - 체력(healthPoints): 몹이 오래 살아남을수록 잡기 어렵기 때문에 난이도에 포함합니다.
    - 공격력(maxDamage): 몹이 주는 피해는 리스크이므로 더 큰 비중(×2)을 둡니다.
    - 따라서 난이도 점수는 생존력 + 위협도를 동시에 반영한 값입니다.

## Join data

In [4]:
# mobID → 몹 ID, foodID → 음식 ID로 연결

# mob_food + food 조인 (foodID 기준)
mob_food_value = mob_food.join(food, mob_food["foodID"] == food["ID"], how="left") \
                         .select(mob_food["mobID"], food["hunger"])

# 몹별 총 음식 드롭 가치 합산
mob_drop_value = mob_food_value.groupBy("mobID").agg(
    spark_sum("hunger").alias("totalDropValue")
)

mob_drop_value.show(10)

+-----+--------------+
|mobID|totalDropValue|
+-----+--------------+
|   53|           3.0|
|   78|           8.0|
|   34|           3.0|
|   28|           2.0|
|   76|           8.0|
|   26|           2.0|
|   22|           3.0|
|   52|           4.0|
|    6|           2.0|
|   54|           3.0|
+-----+--------------+
only showing top 10 rows



- mob_food과 food 데이터를 음식 ID로 연결하여, 몹이 드롭하는 음식의 hunger 점수를 가져왔습니다.
- 연결된 데이터에서 각 몹이 드롭하는 음식의 hunger 점수를 모두 더해 totalDropValue를 계산했습니다.
- 결과는 몹 ID별로 드롭 가치가 얼마인지 보여줍니다.
    - 예: mobID=78은 허기짐을 총 8칸 채울 수 있는 양의 음식을 드롭합니다.

In [5]:
from pyspark.sql.functions import col, when, format_number

# mobs.ID ↔ mob_drop_value.mobID 연결
mob_efficiency = mobs.join(mob_drop_value, mobs["ID"] == mob_drop_value["mobID"], how="left")

# NULL 처리: 드롭 없으면 0
mob_efficiency = mob_efficiency.withColumn(
    "totalDropValue",
    when(col("totalDropValue").isNull(), 0).otherwise(col("totalDropValue"))
)

# 가성비 계산: 드롭 가치 / (사냥 난이도 + 1)
mob_efficiency = mob_efficiency.withColumn(
    "efficiencyScore",
    col("totalDropValue") / (col("difficultyScore") + 1)
)

# totalDropValue가 0인 행 제거
mob_efficiency = mob_efficiency.filter(col("totalDropValue") > 0)

# 소수점 둘째자리로 포맷 (출력용)
mob_efficiency = mob_efficiency.withColumn(
    "efficiencyScore", format_number("efficiencyScore", 2)
)

# 출력
mob_efficiency.select(
    "name", "healthPoints", "maxDamage", "difficultyScore", "totalDropValue", "efficiencyScore"
).orderBy(col("efficiencyScore").desc()).show(10, truncate=False)

+---------------+------------+---------+---------------+--------------+---------------+
|name           |healthPoints|maxDamage|difficultyScore|totalDropValue|efficiencyScore|
+---------------+------------+---------+---------------+--------------+---------------+
|salmon         |3.0         |0.0      |3.0            |2.0           |0.50           |
|cod            |3.0         |0.0      |3.0            |2.0           |0.50           |
|chicken        |4.0         |0.0      |4.0            |2.0           |0.40           |
|zombie_villager|20.0        |3.0      |26.0           |8.0           |0.30           |
|husk           |20.0        |3.0      |26.0           |8.0           |0.30           |
|zombie         |20.0        |3.0      |26.0           |8.0           |0.30           |
|mooshroom      |10.0        |0.0      |10.0           |3.0           |0.27           |
|pig            |10.0        |0.0      |10.0           |3.0           |0.27           |
|cow            |10.0        |0.

- mobs 데이터와 mob_drop_value 데이터를 몹 ID 기준으로 결합합니다.
- 드롭 음식 정보가 없는 몹은 totalDropValue = 0으로 처리합니다.

---

- 가성비 계산: efficiencyScore = totalDropValue / (difficultyScore + 1)
- 드롭 가치가 0인 몹은 분석에서 제외합니다.

In [6]:
from pyspark.sql.functions import col

mob_efficiency = mob_efficiency.withColumn("efficiencyScore", col("efficiencyScore").cast("double"))

## 모델 학습 및 평가

In [7]:
# 피처 엔지니어링: 필요 컬럼 선택
features = ["healthPoints", "maxDamage", "totalDropValue"]

In [8]:
stages = []

In [9]:
from pyspark.ml.feature import StandardScaler, VectorAssembler

num_assembler = VectorAssembler(inputCols=["healthPoints", "maxDamage", "totalDropValue"], outputCol= 'feature_vector')
stages += [num_assembler]

stages

[VectorAssembler_c5fddc12fa8a]

In [10]:
train_df, test_df = mob_efficiency.randomSplit([0.95,0.05], seed=300)

In [11]:
from pyspark.ml import Pipeline
pipeline = Pipeline(stages=stages)
fitted_transform = pipeline.fit(train_df)
vtrain_df = fitted_transform.transform(train_df)
vtrain_df.printSchema()

root
 |-- ID: integer (nullable = true)
 |-- name: string (nullable = true)
 |-- healthPoints: double (nullable = true)
 |-- maxDamage: double (nullable = true)
 |-- difficultyScore: double (nullable = true)
 |-- mobID: integer (nullable = true)
 |-- totalDropValue: double (nullable = true)
 |-- efficiencyScore: double (nullable = true)
 |-- feature_vector: vector (nullable = true)



In [12]:
vtrain_df.select('feature_vector', 'efficiencyScore').show(2)

+--------------+---------------+
|feature_vector|efficiencyScore|
+--------------+---------------+
| [4.0,0.0,2.0]|            0.4|
|[10.0,3.0,2.0]|           0.12|
+--------------+---------------+
only showing top 2 rows



In [13]:
from pyspark.ml.regression import LinearRegression
lr = LinearRegression(maxIter=50, solver='normal', 
                 labelCol='efficiencyScore', featuresCol='feature_vector')

In [14]:
model = lr.fit(vtrain_df)

In [15]:
#테스트데이터도 변환
vtest_df = fitted_transform.transform(test_df)
#테스트데이터로 예측
pred = model.transform(vtest_df)

In [16]:
pred.select('efficiencyScore', 'prediction').show()

+---------------+--------------------+
|efficiencyScore|          prediction|
+---------------+--------------------+
|           0.02|-0.18378511036286294|
|           0.22|  0.3227520020579866|
+---------------+--------------------+



In [17]:
model.summary.r2, model.summary.rootMeanSquaredError

(0.7443854326480102, 0.0698451417899431)

## 결론

전체 데이터를 기반으로 한 선형 회귀 모델의 성능은 기대에 못 미쳤습니다. 몹의 체력(healthPoints)과 공격력(maxDamage)만으로는 드롭 가치를 충분히 설명하기 어려웠습니다.
- 설명력(R²)이 약 0.69 수준으로, 예측 모델이 드롭 가치 변동의 69% 정도만 설명하고 있다는 뜻입니다. 이는 중간 수준의 설명력으로, 현실적으로는 예측 신뢰도가 높지 않다는 해석도 가능합니다.
- 평균 제곱근 오차(RMSE)는 0.0784로 비교적 작게 보일 수 있지만, 전체 점수 분포가 좁은 상황에서는 이 수치도 의미 있는 오차로 간주할 수 있습니다.
- 테스트 세트의 예측 효율 점수 분포를 보면, 실제 점수와 큰 차이를 보이는 몹들이 적지 않으며, 상위권 효율 몹 선정에서도 예측이 엇갈린 경우가 있습니다.

결국, 체력과 공격력만으로 몹의 가성비를 예측하기에는 정보가 부족하다는 것이 확인됐습니다. 몹의 드롭 효율은 난이도, 리스폰 빈도, 이동 패턴, 공격 빈도 등 다양한 요소의 영향을 받기 때문에, 모델에 반영된 피처가 너무 단순했던 것으로 판단됩니다.

향후 분석을 개선하기 위해서는 다음과 같은 보완이 필요합니다:
- 공격 빈도, 사망 시점 조건 등의 난이도 관련 요소 추가
- 드롭 확률 또는 드롭 아이템 다양성 반영
- 범주형 변수(몹 타입, 차원 구분 등) 도입을 통한 구조적 다양성 확보

현재 모델은 단순 구조 기반 예측 실험의 초석이 되었으며, 게임 내 실제 밸런싱 판단에는 부족한 수준임을 확인한 것이 이번 분석의 주요 결론입니다.